In [112]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import scipy.constants as const
import autograd.numpy as anp
import pandas as pd

from autograd import grad

In [113]:
class StateVector:
    def __init__(self, x, y, z, vx, vy, vz, ax, ay, az, m):
        self.r = anp.array([x, y, z], dtype=float)
        for i in range(3):
            if self.r[i] == 0:
                self.r[i] = 1e-10
        self.v = anp.array([vx, vy, vz], dtype=float)
        for i in range(3):
            if self.v[i] == 0:
                self.v[i] = 1e-10  # Avoid zero velocities

        self.a = anp.array([ax, ay, az], dtype=float)
        
        for i in range(3):
            if self.a[i] == 0:
                self.a[i] = 1e-10
        self.m = float(m)
        self.s = anp.array([self.r, self.v, self.a], dtype=float)
        
# Initialize bodies
body_one = StateVector(0, 0, 0, 100, 0, 0, 0, 0, 0, 2)
body_two = StateVector(1, 0, 0, 0, 1, 0, 0, 0, 0, 1)

$$
H = \sum_{i=1}^{N} \frac{1}{2} m_i |\mathbf{v}_i|^2 - \sum_{i=1}^{N} \sum_{j=i+1}^{N} \frac{G m_i m_j}{r_{ij}}
$$

In [114]:

def T(state_vectors):
    masses = anp.array([body.m for body in state_vectors], dtype=float)
    velocities = anp.array([body.v for body in state_vectors], dtype=float)
    T_total = anp.sum(0.5 * masses * anp.linalg.norm(velocities, axis=1) ** 2)
    return T_total

def V(state_vectors):
    masses = anp.array([body.m for body in state_vectors], dtype=float)
    positions = anp.array([body.r for body in state_vectors], dtype=float)
    V_total = 0.0
    num_bodies = len(state_vectors)
    for i in range(num_bodies):
        for j in range(i + 1, num_bodies):
            r_ij = anp.linalg.norm(positions[i] - positions[j])
            V_total -= masses[i] * masses[j] / r_ij
    return V_total
    
def H(state_vectors):

    H_total = T(state_vectors) + V(state_vectors)
    
    return H_total

$$
\dot{\mathbf{r}}_{i, 1}^{\text{Ham}} = \frac{\partial H}{\partial \mathbf{p}_{i, 1}}= \frac{1}{m_i}  \frac{\partial H}{\partial\mathbf{v}_{i, 1}^{\text{pred}}}
$$

In [115]:
def H_velocity_gradient(state_vectors):
    def H_velocities(velocities_flat):
        velocities = velocities_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(body.r[0], body.r[1], body.r[2],
                                          velocities[i, 0], velocities[i, 1], velocities[i, 2],
                                          body.a[0], body.a[1], body.a[2], body.m) 
                              for i, body in enumerate(state_vectors)]
        return H(temp_state_vectors)

    velocities_flat = anp.array([body.v for body in state_vectors], dtype=float).flatten()
    grad_H = grad(H_velocities)
    grad_values = grad_H(velocities_flat).reshape(len(state_vectors), 3)
    masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = np.round(grad_values / masses, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

hamiltonian = H([body_one, body_two])
print("Hamiltonian:", hamiltonian)

gradient = H_velocity_gradient([body_one, body_two])
print("Gradient of Hamiltonian w.r.t velocity:")
print(gradient)

Hamiltonian: 9998.4999999998
Gradient of Hamiltonian w.r.t velocity:
          Gradient X    Gradient Y    Gradient Z
Body 0  1.000000e+02  1.000000e-10  1.000000e-10
Body 1  1.000000e-10  1.000000e+00  1.000000e-10


$$
  \dot{\mathbf{v}}_{i, 1}^{\text{Ham}} = \frac{\dot{\mathbf{p}}_{i, 1}}{m_i} = \frac{-1}{m_i} \frac{\partial H}{\partial \mathbf{r}_{i, 1}^{\text{pred}
  }}
$$

In [116]:
def H_position_gradient(state_vectors):
    def H_positions(positions_flat):
        positions = positions_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(positions[i, 0], positions[i, 1], positions[i, 2],
                                          body.v[0], body.v[1], body.v[2],
                                          body.a[0], body.a[1], body.a[2], body.m) 
                              for i, body in enumerate(state_vectors)]
        return H(temp_state_vectors)

    positions_flat = anp.array([body.r for body in state_vectors], dtype=float).flatten()
    grad_H = grad(H_positions)
    grad_values = grad_H(positions_flat).reshape(len(state_vectors), 3)
    masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = -anp.round(grad_values / masses, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

gradient = H_position_gradient([body_one, body_two])
print("Gradient of Hamiltonian w.r.t position:")
print(gradient)

Gradient of Hamiltonian w.r.t position:
        Gradient X  Gradient Y  Gradient Z
Body 0         1.0        -0.0        -0.0
Body 1        -2.0        -0.0        -0.0


In [120]:
def V_position_gradient(state_vectors):
    def V_positions(positions_flat):
        positions = positions_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(positions[i, 0], positions[i, 1], positions[i, 2],
                                          body.v[0], body.v[1], body.v[2],
                                          body.a[0], body.a[1], body.a[2], body.m) 
                              for i, body in enumerate(state_vectors)]
        return V(temp_state_vectors)

    positions_flat = anp.array([body.r for body in state_vectors], dtype=float).flatten()
    grad_V = grad(V_positions)
    grad_values = grad_V(positions_flat).reshape(len(state_vectors), 3)
    # masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = -anp.round(grad_values, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

gradient = V_position_gradient([body_one, body_two])
print("Gradient of Potential Energy w.r.t position:")
print(gradient)

Gradient of Potential Energy w.r.t position:
        Gradient X  Gradient Y  Gradient Z
Body 0         2.0        -0.0        -0.0
Body 1        -2.0        -0.0        -0.0
